<a href="https://colab.research.google.com/github/IzzulGod/Sorachio-360M-Chat/blob/main/source/sorachio_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenizer

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login

login(token="Lorem-ipsum")


In [ ]:
!wget -q https://raw.githubusercontent.com/damzaky/kumpulan-kata-bahasa-indonesia-KBBI/master/legacy/indonesian-words.txt -O indo_kbbi.txt


In [ ]:
import os, random

# Pastikan file ada dan baca
assert os.path.exists("indo_kbbi.txt"), "File indo_kbbi.txt gak ketemu!"

with open("indo_kbbi.txt", encoding="utf-8") as f:
    all_words = [w.strip() for w in f if w.strip()]

print(f"🔍 Total kata di KBBI list: {len(all_words)}")  # ~40K

# Ambil 10.000 kata acak
random.seed(42)
if len(all_words) >= 10000:
    top_words = random.sample(all_words, 10000)
else:
    top_words = all_words

# Simpan
with open("top_10000_words_indo.txt", "w", encoding="utf-8") as f:
    for w in top_words:
        f.write(w + "\n")

print(f"✅ Sudah tersimpan {len(top_words)} kata di top_10000_words_indo.txt")


🔍 Total kata di KBBI list: 14556
✅ Sudah tersimpan 10000 kata di top_10000_words_indo.txt


In [ ]:
from transformers import AutoTokenizer

# Load tokenizer asli
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")

# Baca kosakata baru dan filter yg belum ada
with open("top_10000_words_indo.txt", encoding="utf-8") as f:
    new_tokens = [w.strip() for w in f if w.strip() and w.strip() not in tokenizer.get_vocab()]

# Tambahkan
num_added = tokenizer.add_tokens(new_tokens)
print(f"✅ Berhasil menambahkan {num_added} token baru ke tokenizer.")

# Simpan tokenizer hasil adaptasi
tokenizer.save_pretrained("tokenizer_smolLM2_indo_adapted")


tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

✅ Berhasil menambahkan 9810 token baru ke tokenizer.


('tokenizer_smolLM2_indo_adapted/tokenizer_config.json',
 'tokenizer_smolLM2_indo_adapted/special_tokens_map.json',
 'tokenizer_smolLM2_indo_adapted/chat_template.jinja',
 'tokenizer_smolLM2_indo_adapted/vocab.json',
 'tokenizer_smolLM2_indo_adapted/merges.txt',
 'tokenizer_smolLM2_indo_adapted/added_tokens.json',
 'tokenizer_smolLM2_indo_adapted/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer

old = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")
new = AutoTokenizer.from_pretrained("tokenizer_smolLM2_indo_adapted")

sent = "kecerdasan buatan di Indonesia sangat berkembang"
print("Sebelum:", old.tokenize(sent))
print("Sesudah:", new.tokenize(sent))


Sebelum: ['ke', 'cer', 'd', 'asan', 'Ġbu', 'atan', 'Ġdi', 'ĠIndonesia', 'Ġsang', 'at', 'Ġber', 'k', 'emb', 'ang']
Sesudah: ['kecerdasan', 'Ġbu', 'atan', 'Ġdi', 'ĠIndonesia', 'Ġ', 'sangat', 'Ġ', 'berkembang']


In [ ]:
!cp -r /content/tokenizer_smolLM2_indo_adapted /content/drive/MyDrive/


# Fine-Tune

In [ ]:
!pip install --quiet transformers peft accelerate
!pip install --quiet datasets
!pip install --upgrade fsspec datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(
    "/content/drive/MyDrive/tokenizer_smolLM2_indo_adapted",
    use_fast=True,
)


model = AutoModelForCausalLM.from_pretrained(
    "HuggingFaceTB/SmolLM2-360M-Instruct",
    device_map="auto",
    torch_dtype="auto",
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
import os

# — Setup cache
cache_dir = "./hf_cache"
os.makedirs(cache_dir, exist_ok=True)

# — Parser untuk Alpaca GPT4 Indo
def load_and_parse_alpaca():
    ds = load_dataset(
        "FreedomIntelligence/alpaca-gpt4-indonesian",
        split="train",
        cache_dir=cache_dir
    )
    def parse(ex):
        conv = ex["conversations"]
        human = next((m["value"] for m in conv if m["from"]=="human"), "").strip()
        gpt   = next((m["value"] for m in conv if m["from"]!="human"), "").strip()
        return {"instruction": human, "output": gpt}
    return ds.map(parse, remove_columns=ds.column_names)

# — Parser untuk ShareGPT Indo
def load_and_parse_sharegpt():
    ds = load_dataset(
        "FreedomIntelligence/sharegpt-indonesian",
        split="train",
        cache_dir=cache_dir
    )
    def parse(ex):
        conv = ex["conversations"]
        human = next((m["value"] for m in conv if m["from"]=="human"), "").strip()
        gpt   = next((m["value"] for m in conv if m["from"]!="human"), "").strip()
        return {"instruction": human, "output": gpt}
    return ds.map(parse, remove_columns=ds.column_names)

# — Parser untuk IndoQA
def load_and_parse_indoqa():
    ds = load_dataset(
        "jakartaresearch/indoqa",
        split="train",
        cache_dir=cache_dir
    )
    def parse(ex):
        ctx = ex.get("context") or ""
        q   = ex.get("question") or ""
        ans = ex.get("answer") or ""
        instr = f"{ctx}\nPertanyaan: {q}".strip()
        return {"instruction": instr, "output": ans.strip()}
    return ds.map(parse, remove_columns=ds.column_names)

# — Load & parse full datasets
ds1 = load_and_parse_alpaca()
ds2 = load_and_parse_sharegpt()
ds3 = load_and_parse_indoqa()

# — Tampilkan 1 sampel dari tiap dataset
print("=== Alpaca GPT4 Indo sample ===")
print(ds1[0])
print("\n=== ShareGPT Indo sample ===")
print(ds2[0])
print("\n=== IndoQA sample ===")
print(ds3[0])


README.md:   0%|          | 0.00/124 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


alpaca-gpt4-indonesian.json:   0%|          | 0.00/50.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49969 [00:00<?, ? examples/s]

Map:   0%|          | 0/49969 [00:00<?, ? examples/s]

...

0000.parquet:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3309 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1104 [00:00<?, ? examples/s]

Map:   0%|          | 0/3309 [00:00<?, ? examples/s]

=== Alpaca GPT4 Indo sample ===
{'instruction': 'Saranlah slogan untuk kampanye daur ulang', 'output': '1. "Kurangi, gunakan kembali, daur ulang: Bersama untuk masa depan yang lebih hijau."\n2. "Daur ulanglah hari ini, untuk masa depan yang lebih baik."\n3. "Ubah sampahmu menjadi harta karun - Daur ulang!"\n4. "Daur ulang untuk siklus kehidupan."\n5. "Simpan sumber daya, daur ulang lebih banyak."'}

=== ShareGPT Indo sample ===
{'instruction': 'Tunjukkan saya daftar pentingnya SEO bagi pemula.', 'output': 'Tentu, berikut adalah daftar dasar-dasar SEO untuk pemula:\n1. Penelitian kata kunci: Ini melibatkan mengidentifikasi kata kunci dan frasa yang dicari oleh audiens target Anda di mesin pencari.\n2. Optimasi halaman: Ini meliputi mengoptimalkan konten situs web Anda, gambar, tag judul, deskripsi meta, dan struktur URL untuk membuatnya lebih bersahabat dengan mesin pencari.\n3. Optimalisasi teknis: Ini melibatkan meningkatkan elemen teknis situs web Anda seperti kecepatan situs, kecoco

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from jinja2 import Template
from datasets import concatenate_datasets
import os

# Load tokenizer dan model
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/tokenizer_smolLM2_indo_adapted", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")
model.resize_token_embeddings(len(tokenizer))

# ChatML template
chatml_tpl = """{% for message in messages %}
<|im_start|>{{ message['role'] }}
{{ message['content'] }}<|im_end|>
{% endfor %}
{% if add_generation_prompt -%}
<|im_start|>assistant
{%- endif %}"""
template = Template(chatml_tpl)

# Build ChatML format dari example
def build_chatml(example):
    instr = example.get("instruction", "").strip()
    output = example.get("output", "").strip()
    if not instr:
        return None
    messages = [
        {"role": "user", "content": instr},
        {"role": "assistant", "content": output}
    ]
    return template.render(messages=messages, add_generation_prompt=True)

# Tokenisasi + label masking
def tokenize_fn(example):
    chatml = build_chatml(example)
    if chatml is None:
        return {}

    encoded = tokenizer(
        chatml,
        truncation=True,
        max_length=1024,
        padding="max_length",
        return_tensors=None,
    )

    # Buat label: input_id normal, pad_token diganti -100
    encoded["labels"] = [
        token if token != tokenizer.pad_token_id else -100
        for token in encoded["input_ids"]
    ]

    encoded["chatml_prompt"] = chatml
    return encoded

ds1 = load_and_parse_alpaca()
ds2 = load_and_parse_sharegpt()
ds3 = load_and_parse_indoqa()
raw_ds = concatenate_datasets([ds1, ds2, ds3])

# Tokenize semua dataset
tokenized = raw_ds.map(
    tokenize_fn,
    remove_columns=raw_ds.column_names,
    batched=False,
    desc="🔄 Tokenizing with ChatML"
)
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print("✅ Total tokenized examples:", len(tokenized))

# Cek hasil decode
def inspect_sample(idx):
    sample = tokenized[idx]
    decoded = tokenizer.decode(sample["input_ids"], skip_special_tokens=False)
    print(f"\n=== 🔍 Sample {idx} ===")
    print(decoded)

# Tes contoh dari tiap dataset
inspect_sample(0)       # Alpaca
inspect_sample(50000)   # ShareGPT
inspect_sample(56000)   # IndoQA

# Statistik token
token_lengths = [len(tokenizer(build_chatml(ex))["input_ids"]) for ex in raw_ds.select(range(100))]
print("📊 Rata-rata panjang token:", sum(token_lengths) / len(token_lengths))
print("🔺 Maksimal panjang token:", max(token_lengths))


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
Repo card metadata block was not found. Setting CardData to empty.


🔄 Tokenizing with ChatML:   0%|          | 0/58838 [00:00<?, ? examples/s]

✅ Total tokenized examples: 58838

=== 🔍 Sample 0 ===

<|im_start|>user
Saranlah slogan untuk kampanye daur ulang<|im_end|>

<|im_start|>assistant
1. "Kurangi, gunakan kembali, daur ulang: Bersama untuk masa depan yang lebih hijau."
2. "Daur ulanglah hari ini, untuk masa depan yang lebih baik."
3. "Ubah sampahmu menjadi harta karun - Daur ulang!"
4. "Daur ulang untuk siklus kehidupan."
5. "Simpan sumber daya, daur ulang lebih banyak."<|im_end|>

<|im_start|>assistant<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|

In [ ]:
sample = tokenized[0]
for inp, lab in zip(sample["input_ids"], sample["labels"]):
    if inp == tokenizer.pad_token_id:
        assert lab == -100
    else:
        assert lab == inp
print("✅ Label mapping valid")


✅ Label mapping valid


In [ ]:
lengths = [len(x["input_ids"]) for x in tokenized]
print("Avg len:", sum(lengths)/len(lengths), "Max len:", max(lengths))


In [ ]:
from peft import LoraConfig, get_peft_model

modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "up_proj", "down_proj", "gate_proj"
]

peft_config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 34,734,080 || all params: 405,972,800 || trainable%: 8.5558


In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="sorachio-indo-finetuned",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=200,
    save_total_limit=1,
    fp16=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model("sorachio-indo-finetuned")


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

# Load base + adapter
base_model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct", device_map="auto")
model = PeftModel.from_pretrained(base_model, "sorachio-indo-finetuned")

# Merge LoRA ke model utama
model = model.merge_and_unload()


In [ ]:
# Save hasil merge
model.save_pretrained("sorachio-indo-merged")
tokenizer.save_pretrained("sorachio-indo-merged")


In [ ]:
!cp -r sorachio-indo-merged /content/drive/MyDrive/
